# 6d reconstruction scan

In [1]:
import os
import sys
sys.path.append("../")

#os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


## define the diagnostic

In [2]:
from measurement import get_DYG14, get_DYG15, set_camera, set_background

save_image_location = "/home/awa/awa_data/12-07_6DReconstruction/reconstruction_scan"
DYG14 = get_DYG14()
DYG15 = get_DYG15()

DYG14.save_image_location = save_image_location
DYG15.save_image_location = save_image_location
DYG14.background_file = save_image_location + "/DYG14_background.npy"
DYG15.background_file = save_image_location + "/DYG15_background.npy"


In [3]:
#set_background(DYG14)
# set the dipole on for this part!
#set_background(DYG15)

### Define the evaluator

In [7]:
from epics import caput, caget
from xopt import Evaluator
import time
import numpy as np

def evaluate_function(inputs: dict, diagnostic, testing=False) -> dict:
    # caput values
    if not testing:
        for name, val in inputs.items():
            caput(name, val)
            print(name, val)

    # wait for changes to occur
    time.sleep(2)
    print(time.time())
    results = diagnostic.measure_beamsize(10, fit_image=False, **inputs)

    return results | inputs

evaluator = Evaluator(function=evaluate_function)

### Set up scan

In [8]:
# quad strength pv values
import numpy as np
import pandas as pd
import yaml
import json

x = np.linspace(-0.5,0.5,21)
quad_pv_name = "AWA:Bira3Ctrl:Ch10"
tdc_pv_name = "pi:QBCR1:Ch7:EnablePulse"
dipole_pv_name = "AWA:Bira4Ctrl:Ch01"
tdc_vals = {"OFF":0, "ON":1} # off/on values
dipole_vals = {"OFF":0, "ON":-5.63} # off/on vals

DYG14.target_charge = 2.5e-9
DYG15.target_charge = 2.5e-9
DYG14.charge_atol = 0.25e-9
DYG15.charge_atol = 0.25e-9
DYG14.wait_time = 0.6
DYG15.wait_time = 0.6


def do_quad_scan(k, pv_dict, diagnostic, testing=False):
    q_scan_data = []
    print("doing quad scan")
    for ele in k:
        # add tcav/dipole settings to quad settings
        total_pvs = pv_dict | {quad_pv_name:ele}
        print(total_pvs)

        # do measurement
        result = evaluate_function(total_pvs, diagnostic, testing)
        q_scan_data += [result]

    return q_scan_data

data = []

# do quad scan with dipole off
set_camera(DYG14)
# set dipole
print("setting dipole")
caput(dipole_pv_name, dipole_vals["OFF"])
time.sleep(15)
print("dipole finished moving")
for tdc_state in ["OFF","ON"]:
    q_scan_results = do_quad_scan(
        x,
        {tdc_pv_name: tdc_vals[tdc_state], dipole_pv_name: dipole_vals["OFF"]},
        DYG14,
    )
    data = data + q_scan_results

    pd_data = pd.DataFrame(data)

    # save dataframe to file
    with open(os.path.join(save_image_location, 'data.yml'), 'w') as outfile:
        yaml.dump(json.loads(pd_data.to_json(orient="records"))
    , outfile, default_flow_style=False)





# do quad scan with dipole on
set_camera(DYG15)
# set dipole
print("setting dipole")
caput(dipole_pv_name, dipole_vals["OFF"])
time.sleep(15)
print("dipole finished moving")

for tdc_state in ["OFF","ON"]:
    q_scan_results = do_quad_scan(
        x,
        {tdc_pv_name: tdc_vals[tdc_state], dipole_pv_name: dipole_vals["ON"]},
        DYG15,
    )
    data = data + q_scan_results

    pd_data = pd.DataFrame(data)

    # save dataframe to file
    with open(os.path.join(save_image_location, 'data.yml'), 'w') as outfile:
        yaml.dump(json.loads(pd_data.to_json(orient="records"))
    , outfile, default_flow_style=False)

pd_data = pd.DataFrame(data)

# save dataframe to file
with open(os.path.join(save_image_location, 'data.yml'), 'w') as outfile:
    yaml.dump(json.loads(pd_data.to_json(orient="records"))
, outfile, default_flow_style=False)

setting camera DYG14
setting dipole
dipole finished moving
doing quad scan
{'pi:QBCR1:Ch7:EnablePulse': 0, 'AWA:Bira4Ctrl:Ch01': 0, 'AWA:Bira3Ctrl:Ch10': -0.5}
pi:QBCR1:Ch7:EnablePulse 0
AWA:Bira4Ctrl:Ch01 0
AWA:Bira3Ctrl:Ch10 -0.5
1701990765.7021666
charge error 4.9e-10 outside atol 2.5e-10
charge error 3.4e-10 outside atol 2.5e-10
charge error 1.6e-09 outside atol 2.5e-10
charge error 3.9e-09 outside atol 2.5e-10
charge error 9e-10 outside atol 2.5e-10
charge error 1.4e-09 outside atol 2.5e-10
charge error 2e-09 outside atol 2.5e-10
charge error 1.8e-09 outside atol 2.5e-10
charge error 5.5e-10 outside atol 2.5e-10
charge error 2.4e-09 outside atol 2.5e-10
charge error 2e-09 outside atol 2.5e-10
charge error 5.2e-10 outside atol 2.5e-10
charge error 1.5e-09 outside atol 2.5e-10
charge error 1.1e-09 outside atol 2.5e-10
charge error 4.4e-10 outside atol 2.5e-10
charge error 1.7e-09 outside atol 2.5e-10
charge error 2.1e-09 outside atol 2.5e-10
charge error 1.4e-09 outside atol 2.5e-10

KeyboardInterrupt: 

CA.Client.Exception...............................................
    Context: "192.168.0.166:5064"
    Source File: ../tcpiiu.cpp line 926
    Current Time: Thu Dec 07 2023 17:54:05.428116133
..................................................................
CA.Client.Exception...............................................
    Context: "192.168.0.167:5064"
    Source File: ../tcpiiu.cpp line 926
    Current Time: Thu Dec 07 2023 17:54:06.930760107
..................................................................
Unexpected problem with CA circuit to server "192.168.0.166:5064" was "No route to host" - disconnecting
CA.Client.Exception...............................................
    Context: "192.168.0.166:5064"
    Source File: ../cac.cpp line 1237
    Current Time: Thu Dec 07 2023 18:09:31.581110244
..................................................................
Unexpected problem with CA circuit to server "192.168.0.167:5064" was "No route to host" - disconnecting
CA.Client

In [6]:
import yaml
import json

